In [3]:
import tweepy
from tweepy import OAuthHandler
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import numpy as np
from collections import Counter
#import nltk
import pandas as pd
import numpy as np
import plotly
from plotly import graph_objs


In [5]:
import re
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)

def preproces(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
def count_by_lambda(expression, word_array):
    return len(list(filter(expression, word_array)))
def count_occurences(character, word_array):
    counter = 0
    for j, word in enumerate(word_array):
        for char in word:
            if char == character:
                counter += 1
    return counter
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# Store the tweets in a dataframe
def process_results(results):
    id_list = [tweet.id for tweet in results]
    data_set = pd.DataFrame(id_list, columns=["id"])
        # Processing Tweet Data

    data_set["text"] = [tweet.text for tweet in results] #text of tweet
    data_set["created_at"] = [tweet.created_at for tweet in results] #when the tweet was created
    data_set["retweet_count"] = [tweet.retweet_count for tweet in results] #number of retweets
    data_set["favorite_count"] = [tweet.favorite_count for tweet in results] #number of favourites
    data_set["source"] = [tweet.source for tweet in results] #source of the tweet
    data_set["length"] = [len(tweet.text) for tweet in results] #number of characters in tweet

    # Processing User Data
    data_set["user_id"] = [tweet.author.id for tweet in results] #id of the author
    data_set["user_screen_name"] = [tweet.author.screen_name for tweet in results] 
    data_set["user_name"] = [tweet.author.name for tweet in results]
    data_set["user_created_at"] = [tweet.author.created_at for tweet in results] #age of user account
    data_set["user_description"] = [tweet.author.description for tweet in results]
    data_set["user_followers_count"] = [tweet.author.followers_count for tweet in results] #number of followers
    data_set["user_friends_count"] = [tweet.author.friends_count for tweet in results] #number of friends
    data_set["user_location"] = [tweet.author.location for tweet in results] #user has a location in profile?
    data_set["user_statuses_count"] = [tweet.author.statuses_count for tweet in results] #number of statuses
    data_set["user_verified"] = [tweet.author.verified for tweet in results] #user is verified?
    data_set["user_url"] = [tweet.author.url for tweet in results] #user has a URL?
    #calculates number of ?, !, hashtags and mentions
    data_set['tokenized_text']= data_set['text'].apply(preproces)
    data_set['token_texts'] = data_set['tokenized_text'].apply(lambda x : [w for w in x if w.lower() not in stop_words])  
    data_set['no_of_question_marks'] = data_set['token_texts'].apply(lambda txt: count_occurences("?", txt)) 
    data_set['no_of_exclamation_marks'] = data_set['token_texts'].apply(lambda txt: count_occurences("!", txt)) 
    data_set['no_of_hashtags'] = data_set['token_texts'].apply(lambda txt: count_occurences("#", txt)) 
    data_set['no_of_mentions'] = data_set['token_texts'].apply(lambda txt: count_occurences("@", txt))
    
    
    #Removes URLs
    data_set['cleaned_text'] = data_set['text'].apply(lambda txt:remove_url_by_regex("http.?://[^\s]+[\s]?",txt))
    #Removes mentions
    data_set['cleaned_text'] = data_set['cleaned_text'].apply(lambda txt:remove_url_by_regex(r'(?:@[\w_]+)',txt))
    #Calculates number of colon marks
    data_set['no_of_colon_marks'] = data_set['cleaned_text'].apply(lambda txt: count_occurences(":", txt)) 
    data_set['cleaned_text'] = data_set['cleaned_text'].apply(lambda txt:remove_url_by_regex(r'[,|:|\|=|&|;|%|$|@|^|*|-|#|?|!|.]',txt))
    data_set['no_of_words'] = data_set['cleaned_text'].apply(lambda txt:len(re.findall(r'\w+',txt)))
    data_set['no_of_uppercase_words'] = data_set['tokenized_text'].apply(lambda txt: count_by_lambda(lambda word: word == word.upper(),txt))
    
    
    return data_set


[nltk_data] Downloading package stopwords to /home/hamdi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def remove_url_by_regex(pattern,string):
    return re.sub(pattern,"", string)

In [25]:
def scrapping(number,text):
  
    consumer_key = '2CF0SjF91irHA3xzSRJwhMiJF'
    consumer_secret = 'ERNMd3a05M39IZgK1B8C6VeIfFauFMxvfY17LxiTD9omlNha0K'
    access_token = '908720407682875393-DkzbECIw8z6Alp3axpLgv3njJ1zcjAU'
    access_secret = 'ZMcWhoSNdYFUwT8aItpYrIg95cGJgYD2zfjerxhqfsstu'

    
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
 
    api = tweepy.API(auth)
    results = []
    i=0
    for tweet in tweepy.Cursor(api.search, q=text, lang="en").items():
        if (not tweet.retweeted) and ('RT @' not in tweet.text):
            results.append(tweet)
            id = tweet.id
            i=i+1
            if i==number:
                break
    #print ("we did a scrapping of"+len(results+" tweet"))
    df = process_results(results)
    df['tokenized_text']= df['text'].apply(preproces)
    df["user_has_url?"] = np.where(df["user_url"].isnull(), '0', '1')
    
    df['user_verified'] = df['user_verified']*1
    df['created_at'].fillna('00-00-0000 00:00', inplace=True)
    df['user_description'].fillna('Unknown_descr', inplace=True)
    df['user_location'].fillna('Unknown_location', inplace=True)
    df['user_statuses_count'].fillna('Unknown', inplace=True)
    df['user_verified'].fillna('0', inplace=True)
    df['user_url'].fillna('Unknown_url', inplace=True)
    df["created_at"]=df["created_at"].apply(lambda x: str(x))
    df["user_created_at"]=df["user_created_at"].apply(lambda x: str(x))
    df['user_has_url?'] = df['user_has_url?'].map( {'1':1, '0':0} )
    df["user_id"]=df["user_id"].apply(lambda x: float(x))
    df=df[["text","created_at","retweet_count","favorite_count","source","length","user_id","user_screen_name","user_name","user_created_at","user_description","user_followers_count","user_friends_count","user_location","user_statuses_count","user_verified","user_url","tokenized_text","token_texts","no_of_question_marks","no_of_exclamation_marks","no_of_hashtags","no_of_mentions","cleaned_text","no_of_colon_marks","no_of_words","no_of_uppercase_words","user_has_url?"]]

    df.dropna(how='any',axis=0,inplace=True) 
    return df

In [26]:
df=scrapping(50,"kais said")
